# ProspAI - MVP algoritmo de predição

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE

In [2]:
url = '../data/dataset_prospai.xlsx'
df = pd.read_excel(url)

### Análise dos Dados 


In [3]:
df.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem,Tipo de Campanha Mais Eficaz,Canal Preferido
0,Feminino,45-54,Superior Completo,Sudeste,Normal/Meio-termo,Campanhas de Desconto,Email
1,Feminino,35-44,Superior Completo,Nordeste,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
2,Masculino,35-44,Superior Incompleto,Norte,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
3,Outro,45-54,Fundamental Incompleto,Centro-Oeste,Normal/Meio-termo,Campanhas de Referência,SMS
4,Feminino,65+,Superior Completo,Norte,Formal,Campanhas de Retenção,SMS


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Gênero                        3000 non-null   object
 1   Faixa Etária                  3000 non-null   object
 2   Escolaridade                  3000 non-null   object
 3   Localização                   3000 non-null   object
 4   Linguagem                     3000 non-null   object
 5   Tipo de Campanha Mais Eficaz  3000 non-null   object
 6   Canal Preferido               3000 non-null   object
dtypes: object(7)
memory usage: 164.2+ KB


In [5]:
df.nunique()

Gênero                          3
Faixa Etária                    7
Escolaridade                    7
Localização                     5
Linguagem                       3
Tipo de Campanha Mais Eficaz    5
Canal Preferido                 5
dtype: int64

### Limpeza e Organização dos Dados

In [6]:
df.rename(columns={'Tipo de Campanha Mais Eficaz': 'Campanha Preferida'}, inplace=True)
df.rename(columns={'Linguagem': 'Linguagem Preferida'}, inplace=True)

In [7]:
df_clean = df

In [8]:
df.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem Preferida,Campanha Preferida,Canal Preferido
0,Feminino,45-54,Superior Completo,Sudeste,Normal/Meio-termo,Campanhas de Desconto,Email
1,Feminino,35-44,Superior Completo,Nordeste,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
2,Masculino,35-44,Superior Incompleto,Norte,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
3,Outro,45-54,Fundamental Incompleto,Centro-Oeste,Normal/Meio-termo,Campanhas de Referência,SMS
4,Feminino,65+,Superior Completo,Norte,Formal,Campanhas de Retenção,SMS


In [9]:
maping = {
    'Masculino' : 0,
    'Feminino': 1,
    'Outro': 2
}
df_clean['Gênero'] = df_clean['Gênero'].replace(maping)

maping2 = {
    '0-17' : 0,
    '18-24': 1,
    '25-34': 2,
    '35-44': 3,
    '45-54': 4,
    '55-64': 5,
    '65+': 6
}
df_clean['Faixa Etária'] = df_clean['Faixa Etária'].replace(maping2)

maping = {
    'Fundamental Incompleto' : 0,
    'Fundamental Completo': 1,
    'Médio Incompleto': 2,
    'Médio Completo': 3,
    'Superior Incompleto': 4,
    'Superior Completo': 5,
    'Pós-graduação': 6
}
df_clean['Escolaridade'] = df_clean['Escolaridade'].replace(maping)

maping = {
    'Sul' : 0,
    'Sudeste': 1,
    'Centro-Oeste': 2,
    'Norte': 3,
    'Nordeste': 4
}
df_clean['Localização'] = df_clean['Localização'].replace(maping)

maping = {
    'Informal': 0,
    'Normal/Meio-termo': 1,
    'Formal': 2
}
df_clean['Linguagem Preferida'] = df_clean['Linguagem Preferida'].replace(maping)

maping = {
    'Campanhas de Desconto': 0,
    'Campanhas de Novos Produtos': 1,
    'Campanhas de Referência': 2,
    'Campanhas de Retenção': 3,
    'Programas de Fidelidade': 4
}
df_clean['Campanha Preferida'] = df_clean['Campanha Preferida'].replace(maping)

maping = {
    'Email': 0,
    'WhatsApp': 1,
    'SMS': 2,
    'Redes Sociais': 3,
    'App Notificações': 4
}
df_clean['Canal Preferido'] = df_clean['Canal Preferido'].replace(maping)

C:\Users\Agatha Pires\AppData\Local\Temp\ipykernel_4816\1365699776.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['Gênero'] = df_clean['Gênero'].replace(maping)
C:\Users\Agatha Pires\AppData\Local\Temp\ipykernel_4816\1365699776.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['Faixa Etária'] = df_clean['Faixa Etária'].replace(maping2)
C:\Users\Agatha Pires\AppData\Local\Temp\ipykernel_4816\1365699776.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ver

In [10]:
df_clean.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem Preferida,Campanha Preferida,Canal Preferido
0,1,4,5,1,1,0,0
1,1,3,5,4,1,1,2
2,0,3,4,3,1,1,2
3,2,4,0,2,1,2,2
4,1,6,5,3,2,3,2


### Classificação | Treinando o modelo

In [17]:
X = df[['Gênero', 'Faixa Etária', 'Escolaridade', 'Localização']]
y = df[['Campanha Preferida', 'Canal Preferido', 'Linguagem Preferida']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'SMOTE' is not defined

In [12]:
xgb_model = xgb.XGBClassifier()
multi_target_model = MultiOutputClassifier(xgb_model, n_jobs=-1)

multi_target_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...),
                      n_jobs=-1)

In [15]:
y_pred = multi_target_model.predict(X_test)

accuracy_campanha = accuracy_score(y_test['Campanha Preferida'], y_pred[:, 0])
accuracy_canal = accuracy_score(y_test['Canal Preferido'], y_pred[:, 1])
accuracy_linguagem = accuracy_score(y_test['Linguagem Preferida'], y_pred[:, 2])

print(f'Acurácia Campanha Preferida: {accuracy_campanha}')
print(f'Acurácia Canal Preferido: {accuracy_canal}')
print(f'Acurácia Linguagem Preferida: {accuracy_linguagem}')

Acurácia Campanha Preferida: 0.21333333333333335
Acurácia Canal Preferido: 0.19
Acurácia Linguagem Preferida: 1.0


c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py", line 213, in accuracy_score
    y_type, y_true, y_pred

Média da acurácia com validação cruzada: nan


c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Agatha Pires\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py", line 213, in accuracy_score
    y_type, y_true, y_pred